In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
import statsmodels.api as sm
from patsy import dmatrices
import nltk
import re
from nltk.corpus import stopwords
from collections import Counter
from sklearn.manifold import MDS
import nltk
#nltk.download('punkt')
from collections import defaultdict
import requests
from bs4 import BeautifulSoup
#import seaborn as sns
import time
import warnings
warnings.filterwarnings("ignore")
from sklearn.cluster import KMeans
from scipy.spatial import ConvexHull
import matplotlib.pyplot as plt

# Loading of Datasets

Dataset with top products and ratings

In [31]:
df_main=pd.read_csv('/Users/advaith/Desktop/MSBA Related coursework/Fall Term/Unstructured Data Analytics/Project/Final Datasets/List of Perfumes to scrape.csv')

In [29]:
df_main

,Name,Gender,Rating Value,Rating Count,Main Accords,Description,url
0,Le Male Le Parfum Jean Paul Gaultierfor men,for men,4.60,13406,"['warm spicy', 'vanilla', 'lavender', 'aromati...",Le Male Le ParfumbyJean Paul Gaultieris a Ambe...,https://www.fragrantica.com/perfume/Jean-Paul-...
1,Emporio Armani Stronger With You Intensely Gio...,for men,4.57,9587,"['vanilla', 'sweet', 'amber', 'cinnamon', 'war...",Emporio Armani Stronger With You IntenselybyGi...,https://www.fragrantica.com/perfume/Giorgio-Ar...
2,Le Beau Le Parfum Jean Paul Gaultierfor men,for men,4.56,7651,"['sweet', 'woody', 'coconut', 'amber', 'vanill...",Le Beau Le ParfumbyJean Paul Gaultieris a Ambe...,https://www.fragrantica.com/perfume/Jean-Paul-...
3,Le Male Elixir Jean Paul Gaultierfor men,for men,4.56,10388,"['vanilla', 'sweet', 'honey', 'amber', 'aromat...",Le Male ElixirbyJean Paul Gaultieris a Amber F...,https://www.fragrantica.com/perfume/Jean-Paul-...
4,Dior Homme Parfum Diorfor men,for men,4.55,7676,"['iris', 'woody', 'leather', 'powdery', 'viole...",Dior Homme ParfumbyDioris a Leather fragrance ...,https://www.fragrantica.com/perfume/Dior/Dior-...
...,...,...,...,...,...,...,...
57,MillÃ©sime ImpÃ©rial Creedfor women and men,for women and men,4.24,"5,616","['marine', 'citrus', 'salty', 'fruity', 'aroma...",MillÃ©sime ImpÃ©rialbyCreedis a Woody Floral M...,https://www.fragrantica.com/perfume/Creed/Mill...
58,Wood Sage & Sea Salt Jo Malone Londonfor women...,for women and men,4.24,"10,208","['aromatic', 'salty', 'marine', 'citrus', 'mus...",Wood Sage & Sea SaltbyJo Malone Londonis a Aro...,https://www.fragrantica.com/perfume/Jo-Malone-...
59,By the Fireplace Maison Martin Margielafor wom...,for women and men,4.24,"15,275","['woody', 'vanilla', 'balsamic', 'warm spicy',...",By the FireplacebyMaison Martin Margielais a W...,https://www.fragrantica.com/perfume/Maison-Mar...
60,Chergui Serge Lutensfor women and men,for women and men,4.24,"10,818","['amber', 'sweet', 'tobacco', 'honey', 'powder...",CherguibySerge Lutensis a Amber Spicy fragranc...,https://www.fragrantica.com/perfume/Serge-Lute...


Dataset with scraped products and corresponding reviews 

In [32]:
df_top = pd.read_csv('/Users/advaith/Desktop/MSBA Related coursework/Fall Term/Unstructured Data Analytics/Project/Final Datasets/CSV_Bestsellers_reviews_corpus.csv')

In [30]:
df_top

,URL,Title,Review
0,https://www.fragrantica.com/perfume/Jean-Paul-...,Le Male Elixir Jean Paul Gaultier cologne - a ...,if u ever wanted to smell like a honey backwoo...
1,https://www.fragrantica.com/perfume/Jean-Paul-...,Le Male Elixir Jean Paul Gaultier cologne - a ...,Very easy to get compliments with Le Male Elix...
2,https://www.fragrantica.com/perfume/Jean-Paul-...,Le Male Elixir Jean Paul Gaultier cologne - a ...,sometimes people need to learn how to take a joke
3,https://www.fragrantica.com/perfume/Jean-Paul-...,Le Male Elixir Jean Paul Gaultier cologne - a ...,smells like sexy honey!! thats best way to des...
4,https://www.fragrantica.com/perfume/Jean-Paul-...,Le Male Elixir Jean Paul Gaultier cologne - a ...,Kind of just a slightly sweeter version of Le ...
...,...,...,...
10495,https://www.fragrantica.com/perfume/Tom-Ford/T...,Tobacco Vanille Tom Ford perfume - a fragrance...,After wearing the delicious Tobacco Vanille sa...
10496,https://www.fragrantica.com/perfume/Tom-Ford/T...,Tobacco Vanille Tom Ford perfume - a fragrance...,So what can I say that hasn't already been sai...
10497,https://www.fragrantica.com/perfume/Tom-Ford/T...,Tobacco Vanille Tom Ford perfume - a fragrance...,"To me, this smells exactly like the waft of sy..."
10498,https://www.fragrantica.com/perfume/Tom-Ford/T...,Tobacco Vanille Tom Ford perfume - a fragrance...,"I realize this review will be unpopular, but h..."


Top attributes obtained

In [15]:
df_top_att = pd.read_csv('/Users/advaith/Desktop/MSBA Related coursework/Fall Term/Unstructured Data Analytics/Project/Final Datasets/Best Attributes.csv')

Attributes related to worst perfumes

In [19]:
df_bottom_att=pd.read_csv('/Users/advaith/Desktop/MSBA Related coursework/Fall Term/Unstructured Data Analytics/Project/Final Datasets/Worst Attributes.csv')

<br>

# Analysis

Joining df_main and df_top

In [35]:
df_combined = pd.merge(df_main, df_top, left_on='url', right_on='URL', how='inner')
df_combined.head()

,Name,Gender,Rating Value,Rating Count,Main Accords,Description,url,URL,Title,Review
0,Emporio Armani Stronger With You Intensely Gio...,for men,4.57,9587,"['vanilla', 'sweet', 'amber', 'cinnamon', 'war...",Emporio Armani Stronger With You IntenselybyGi...,https://www.fragrantica.com/perfume/Giorgio-Ar...,https://www.fragrantica.com/perfume/Giorgio-Ar...,Emporio Armani Stronger With You Intensely Gio...,"Got a tester bottle, DM for inquiries"
1,Emporio Armani Stronger With You Intensely Gio...,for men,4.57,9587,"['vanilla', 'sweet', 'amber', 'cinnamon', 'war...",Emporio Armani Stronger With You IntenselybyGi...,https://www.fragrantica.com/perfume/Giorgio-Ar...,https://www.fragrantica.com/perfume/Giorgio-Ar...,Emporio Armani Stronger With You Intensely Gio...,"Smells like Mexican candy, I fucking love this..."
2,Emporio Armani Stronger With You Intensely Gio...,for men,4.57,9587,"['vanilla', 'sweet', 'amber', 'cinnamon', 'war...",Emporio Armani Stronger With You IntenselybyGi...,https://www.fragrantica.com/perfume/Giorgio-Ar...,https://www.fragrantica.com/perfume/Giorgio-Ar...,Emporio Armani Stronger With You Intensely Gio...,Emporio Armaniâ€™s Stronger With You Intensely...
3,Emporio Armani Stronger With You Intensely Gio...,for men,4.57,9587,"['vanilla', 'sweet', 'amber', 'cinnamon', 'war...",Emporio Armani Stronger With You IntenselybyGi...,https://www.fragrantica.com/perfume/Giorgio-Ar...,https://www.fragrantica.com/perfume/Giorgio-Ar...,Emporio Armani Stronger With You Intensely Gio...,"NON BIASED / NO BULLSHIT REVIEW:Ok, so Iâ€™ve ..."
4,Emporio Armani Stronger With You Intensely Gio...,for men,4.57,9587,"['vanilla', 'sweet', 'amber', 'cinnamon', 'war...",Emporio Armani Stronger With You IntenselybyGi...,https://www.fragrantica.com/perfume/Giorgio-Ar...,https://www.fragrantica.com/perfume/Giorgio-Ar...,Emporio Armani Stronger With You Intensely Gio...,I enjoyed Stronger With You Intenselyâ€”it sme...


Performing Sentiment Analysis

In [36]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

# Initialize the VADER sentiment analyzer
sid = SentimentIntensityAnalyzer()

# Define a function to calculate sentiment scores
def get_sentiment(review):
    sentiment = sid.polarity_scores(review)
    return sentiment['compound']  # 'compound' gives the overall sentiment score

# Apply the sentiment analysis function to each review in the 'Review' column
df_combined['Sentiment_Score'] = df_combined['Review'].apply(get_sentiment)

# Calculate the average sentiment for each product
average_sentiment_per_product = df_combined.groupby('Name')['Sentiment_Score'].mean().reset_index()

# Display the average sentiment for each product
average_sentiment_per_product.head()

,Name,Sentiment_Score
0,02 L'Air du Desert Marocain Tauer Perfumesfor ...,0.659461
1,9pm Afnanfor men,0.539482
2,A*Men Pure Malt Muglerfor men,0.629792
3,Acqua di Parma Blu Mediterraneo - Fico di Amal...,0.666561
4,Allure Eau de Parfum Chanelfor women,0.731841


Combining everything up into one dataframe

In [38]:
product_info = df_combined[['Name', 'Rating Value', 'Main Accords']].drop_duplicates(subset=['Name'])

df_final = pd.merge(average_sentiment_per_product, product_info, on='Name', how='inner')

df_final.head()

,Name,Sentiment_Score,Rating Value,Main Accords
0,02 L'Air du Desert Marocain Tauer Perfumesfor ...,0.659461,4.40,"['woody', 'aromatic', 'amber', 'fresh spicy', ..."
1,9pm Afnanfor men,0.539482,4.50,"['vanilla', 'amber', 'warm spicy', 'cinnamon',..."
2,A*Men Pure Malt Muglerfor men,0.629792,4.45,"['fruity', 'whiskey', 'woody', 'sweet', 'earth..."
3,Acqua di Parma Blu Mediterraneo - Fico di Amal...,0.666561,4.32,"['citrus', 'woody', 'fruity', 'sweet', 'fresh ..."
4,Allure Eau de Parfum Chanelfor women,0.731841,4.13,"['citrus', 'floral', 'white floral', 'vanilla'..."


Assigning scores to top notes

In [48]:
top_10_notes = df_top_att['Notes'].head(10).tolist()
top_note_scores = {note: score for score, note in enumerate(reversed(top_10_notes))}
top_note_scores

{'citrus': 0,
 'fresh': 1,
 'warm': 2,
 'tobacco': 3,
 'floral': 4,
 'leather': 5,
 'wood': 6,
 'intense': 7,
 'vanilla': 8,
 'sweet': 9}

Assigning scores to notes from bad notes

In [49]:
bottom_10_notes = df_bottom_att['Notes'].head(10).tolist()
bottom_note_scores = {note: score for score, note in enumerate(reversed(bottom_10_notes))}
bottom_note_scores

{'cedar': 0,
 'patchouli': 1,
 'sandalwood': 2,
 'vanilla': 3,
 'rose': 4,
 'jasmine': 5,
 'bergamot': 6,
 'orange': 7,
 'musk': 8,
 'amber': 9}

In [60]:
# Function to evaluate the main accords based on the top and bottom notes
def evaluate_main_accords(main_accords, top_scores, bottom_scores):
    score = 0
    for accord in main_accords.split(","):  # Split main accords by commas
        accord = accord.strip().lower()  # Clean and lowercase each accord
        if accord in top_scores:
            score += top_scores[accord]  # Add positive score for top notes
        elif accord in bottom_scores:
            score -= bottom_scores[accord]  # Subtract (penalize) for bottom notes
        else:
            score += 0  # Assign a value of zero for accords not in the top or bottom lists
    return score

# Apply the function to the 'Main Accords' column in df_final
df_final['Evaluation_Score'] = df_final['Main Accords'].apply(evaluate_main_accords, 
                                                              top_scores=top_note_scores, 
                                                              bottom_scores=note_scores)

# Display the updated df_final with the new Evaluation Score column
df_final.head(10)

,Name,Sentiment_Score,Rating Value,Main Accords,Evaluation_Score
0,02 L'Air du Desert Marocain Tauer Perfumesfor ...,0.659461,4.40,"['woody', 'aromatic', 'amber', 'fresh spicy', ...",0
1,9pm Afnanfor men,0.539482,4.50,"['vanilla', 'amber', 'warm spicy', 'cinnamon',...",0
2,A*Men Pure Malt Muglerfor men,0.629792,4.45,"['fruity', 'whiskey', 'woody', 'sweet', 'earth...",0
3,Acqua di Parma Blu Mediterraneo - Fico di Amal...,0.666561,4.32,"['citrus', 'woody', 'fruity', 'sweet', 'fresh ...",0
4,Allure Eau de Parfum Chanelfor women,0.731841,4.13,"['citrus', 'floral', 'white floral', 'vanilla'...",0
5,Ani Nishanefor women and men,0.572985,4.28,"['warm spicy', 'vanilla', 'woody', 'citrus', '...",0
6,Baccarat Rouge 540 Extrait de Parfum Maison Fr...,0.574115,4.26,"['woody', 'almond', 'amber', 'warm spicy', 'an...",0
7,Bottega Veneta Bottega Venetafor women,0.663031,4.17,"['leather', 'earthy', 'mossy', 'patchouli', 'w...",0
8,Brazilian Crush Cheirosa '62 Sol de Janeirofor...,0.617646,4.26,"['vanilla', 'caramel', 'sweet', 'almond', 'nut...",0
9,By the Fireplace Maison Martin Margielafor wom...,0.673999,4.24,"['woody', 'vanilla', 'balsamic', 'warm spicy',...",0
